In [81]:
import pandas as pd 

not_tr = pd.read_csv('not_tr.csv')
not_tr_urls = not_tr['urls'].tolist()
print(len(not_tr_urls)) 

5733


In [ ]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument('disable-gpu')
options.add_argument('User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36')
options.add_argument('lang=ko_KR')

# /Users/baeyuna/Documents/Crawling/xpath.ipynb
chromedriver = '/Users/baeyuna/Documents/chromedriver'
driver = webdriver.Chrome(chromedriver, options=options)

In [ ]:
import time
from random import *
from tqdm import tqdm

dates = []
MAX_SLEEP_TIME = 3

for url in tqdm(not_tr_urls):
    rand_value = randint(1, MAX_SLEEP_TIME)
    time.sleep(rand_value)
    driver.get(url)
    
    try:
        datetime = driver.find_element_by_xpath('//*[@id="precontent"]/section/div[1]/div/div/span[3]/time')
        date = datetime.get_attribute('datetime')
        dates.append(date)
    except:
        dates.append(None)

df = pd.DataFrame({'date':dates, 'url':urls})
df

 40%|████      | 2314/5733 [2:13:47<3:43:38,  3.92s/it] 

In [ ]:
print('잘 뽑힌 값 개수:',df.loc[~df['date'].isnull()].shape)

# 데이터 합치기

In [ ]:
# 기존 크롤링한 데이터 
msn_train= pd.read_csv('/Users/baeyuna/Documents/SNU_DLab/Data/msn_train.csv')

In [ ]:
# 데모 데이터 

demo_train = pd.read_csv('/Users/baeyuna/Documents/SNU_DLab/Data/MINDdemo_train/news.tsv', delimiter='\t', header=None)
print(demo_train.shape)
demo_train.head()

In [ ]:
# 기존 수집된 데이터로 붙이기
demo_train_nid = demo_train.iloc[:,0]

demo_trains = msn_train.loc[msn_train.iloc[:,6].isin(demo_train_nid)]
print(demo_trains.shape)
demo_trains.head()

In [ ]:
# 추가 수집 데이터 
titles = []   
contents = []
images = [] 


print('crawling start.')
start_time = int(time.time())

url_except =[]
num = 0
total = len(not_train_urls)
for url in not_train_urls:
    # 진행상황 확인 
    num += 1
    if num % 2000 == 0:
        print('{} urls are crawled.\n {} are remained'.format(num, total - num))

    try:
        req = requests.get(url)
        html = req.text
        soup = BeautifulSoup(html, 'html.parser')


        # title
        if soup.find('h1').text.strip() is not None:
            title = soup.find('h1').text.strip()
            titles.append(title)
        elif soup.find('header').text.strip() is not None:
            title = soup.find('header').text.strip()
            titles.append(title)
        else:
            titles.append(None)

        # content
        if soup.find_all('p') is not None:
            content = []
            for el in soup.find_all('p'):
                content.append(el.get_text()) 
            contents.append(' '.join(content))
        elif soup.find('h2').text.strip() is not None:
            content = soup.find('h2').text.strip()
            contents.append(content)
        else:
            contents.append(url)

        # image
        try:
            image = soup.find('img').get('src')
            images.append(image)
        except:
            images.append('no image')


        # 길이 맞추기 
        if len(url_lst) != num:
            url_lst.append(None)
        if len(titles) != num:
            titles.append(None)
        if len(contents) != num:
            contents.append(None)
        if len(images) != num:
            images.append(None)

    except:
        url_except.append(url)

print('-------Seconds: %s--------' % (time.time() - start_time))
print('-------url_except-----')
print(len(url_except)) 

print('len titles', len(titles))
print('len dates', len(dates))
print('len contents', len(contents))
print('len images', len(images))

In [ ]:
df_train = pd.DataFrame({'url' : not_train_urls,
                   'Title':titles,
                   'Date': dates, 
                   'Content': contents,
                   'Image': images})
print(df_train.shape)

import numpy as np

for i in range(len(df_train)):
    if df_train.loc[i, 'Title'] == 'Whoops!':
        df_train.loc[i, 'Title'] = np.nan 
        df_train.loc[i, 'Date'] = np.nan 
        df_train.loc[i, 'Content'] = np.nan 
        df_train.loc[i, 'Image'] = np.nan 

print('잘 뽑힌 값 개수:',df_train.loc[~df_train['date'].isnull()].shape)
df_train.head()

In [ ]:
# 데이터 붙이기

print(demo_trains.shape)
print(df_train.shape)
print(pd.concat([demo_trains,df_train]).shape)

In [ ]:
df_train.rename(columns = {'url' : 5}, inplace = True)

demo_train = pd.merge(not_train, df_train, how='inner',on= 5)
print(demo_train.shape)

print('-----columns-----')
print(demo_trains.columns.tolist())
demo_train.head()

In [ ]:
demo_train = demo_train.iloc[:,[5,8,9,11,10,0,1,2,3,4,6,7]]
print(demo_train.columns.tolist()) 

demo_train.rename(columns = {5 : '5'}, inplace = True)
demo_train.rename(columns = {0 : '0'}, inplace = True)
demo_train.rename(columns = {1 : '1'}, inplace = True)
demo_train.rename(columns = {2 : '2'}, inplace = True)
demo_train.rename(columns = {3 : '3'}, inplace = True)
demo_train.rename(columns = {4 : '4'}, inplace = True)
demo_train.rename(columns = {6 : '6'}, inplace = True)
demo_train.rename(columns = {7 : '7'}, inplace = True)

In [ ]:
demo_trains = demo_trains.drop(['nids'], axis=1)
print(demo_trains.columns.tolist())

demo = pd.concat([demo_train, demo_trains])
print(demo.shape)

demo.to_csv('integrated_news.csv', index=False)
demo.head()